In [1]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
import senteval


In [2]:

model_path = tokenizer_path = "bert-base-uncased"  
model = BertForSequenceClassification.from_pretrained(model_path, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [4]:
import numpy as np
import senteval
def prepare(params, samples):
    return 

def batcher(_, batch):
    batch = [" ".join(sts) for sts in batch]
    inputs = tokenizer(batch, return_tensors="pt",
            add_special_tokens=True,
            pad_to_max_length=True,         # Pad sentence to max length
            truncation=True)
    for k in inputs:
        inputs[k] = inputs[k].to("cuda")
    res = model(**inputs)["hidden_states"]
    res = res[-1][:, 0].detach().cpu().numpy()
    return res


device='cuda'

model = model.to(device)
PATH_TO_DATA = "SentEval/data"
PARAMS = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 10, 'batch_size': 128}
PARAMS['classifier'] = {'max_itaser': 10000, 'nhid': 768}
transfer_tasks = ['CR', 'MR', 'MPQA', 'SUBJ', 'SST2', 'SST5', 'TREC', 'MRPC', 'SNLI', 'SICKEntailment', 'SICKRelatedness', 'STSBenchmark']
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    se = senteval.engine.SE(PARAMS, batcher, prepare)
    results = se.eval(transfer_tasks)
# print('Time took on task %s : %.1f. seconds' % (TASK, end - start))
print(results)

{'CR': {'devacc': 87.36, 'acc': 87.6, 'ndev': 3775, 'ntest': 3775}, 'MR': {'devacc': 82.42, 'acc': 82.53, 'ndev': 10662, 'ntest': 10662}, 'MPQA': {'devacc': 88.41, 'acc': 88.34, 'ndev': 10606, 'ntest': 10606}, 'SUBJ': {'devacc': 95.64, 'acc': 95.7, 'ndev': 10000, 'ntest': 10000}, 'SST2': {'devacc': 86.24, 'acc': 86.77, 'ndev': 872, 'ntest': 1821}, 'SST5': {'devacc': 46.23, 'acc': 45.52, 'ndev': 1101, 'ntest': 2210}, 'TREC': {'devacc': 86.79, 'acc': 91.8, 'ndev': 5452, 'ntest': 500}, 'MRPC': {'devacc': 69.16, 'acc': 70.09, 'f1': 79.18, 'ndev': 4076, 'ntest': 1725}, 'SNLI': {'devacc': 61.88, 'acc': 61.89, 'ndev': 9842, 'ntest': 9824}, 'SICKEntailment': {'devacc': 73.0, 'acc': 71.28, 'ndev': 500, 'ntest': 4927}, 'SICKRelatedness': {'devpearson': 0.7012727576920597, 'pearson': 0.6963877554642018, 'spearman': 0.6423404595047993, 'mse': 0.5256912964540551, 'yhat': array([3.21720952, 4.01886619, 2.35996623, ..., 3.29405314, 4.38710526,
       4.20570849]), 'ndev': 500, 'ntest': 4927}, 'STSBen